In [32]:
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn
# !pip install tensorflow
# !pip install scikit-learn

# Taller 2: Implementación de Machine Learning en un Supermercado Inteligente

### Estudiante: Diego Alvaro Morales Medrano
### Código: 202315708

## Introducción

El objetivo de este taller es desarrollar un modelo de Machine Learning que permita identificar productos en un supermercado inteligente. Esto facilitará a los clientes tomar productos y salir sin pasar por caja, ya que las cámaras y sensores registrarán automáticamente los artículos seleccionados.

En este taller, abordaremos los siguientes puntos:

1. Entendimiento y preparación de los datos.
2. Entrenamiento de dos modelos de Machine Learning.
3. Análisis de resultados y selección del mejor modelo.
4. Cálculo del valor generado y ROI.
5. Presentación de insights y recomendaciones.
6. Clasificación detallada por tipos de productos y sus marcas.

## 1. Entendimiento y Preparación de los Datos

### 1.1 Importación de Librerías

In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense,
                                     Dropout, BatchNormalization)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from PIL import Image

### 1.2 Descarga y Preparación del Dataset

In [1]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

Cloning into 'GroceryStoreDataset'...
remote: Enumerating objects: 6559, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 6559 (delta 45), reused 37 (delta 35), pack-reused 6293 (from 1)
Receiving objects: 100% (6559/6559), 116.26 MiB | 1.30 MiB/s, done.
Resolving deltas: 100% (275/275), done.
Updating files: 100% (5717/5717), done.


### 1.3 Cargando el Dataset

El dataset se encuentra en la carpeta `GroceryStoreDataset` y está estructurado de la siguiente manera:

- `train.txt`, `val.txt`, `test.txt`: Archivos que contienen las rutas de las imágenes y sus etiquetas.
- `classes.csv`: Archivo que contiene los nombres de las clases y sus etiquetas correspondientes.

Vamos a cargar estos archivos y explorar el dataset.

In [35]:
# Ruta al directorio de datos
data_dir = './GroceryStoreDataset/dataset'

# Cargar clases
classes_df = pd.read_csv(os.path.join(data_dir, 'classes.csv'))
classes_df.columns = ['class_name', 'class_id', 'coarse_class_name', 'coarse_class_id', 'class_image', 'product_description']

# Crear diccionarios para mapear etiquetas a nombres
fine_label_to_name = dict(zip(classes_df['class_id'], classes_df['class_name']))
coarse_label_to_name = dict(zip(classes_df['coarse_class_id'], classes_df['coarse_class_name']))

### 1.4 Exploración de Clases

In [36]:
# Función para cargar los datos desde los archivos txt
def load_data(txt_file):
    data = []
    with open(os.path.join(data_dir, txt_file), 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split(",")
            image_path = parts[0]
            fine_label = int(parts[1])
            coarse_label = int(parts[2])
            data.append({'image_path': os.path.join(data_dir, image_path),
                         'fine_label': fine_label,
                         'coarse_label': coarse_label})
    return pd.DataFrame(data)

# Cargar conjuntos de datos
train_df = load_data('train.txt')
val_df = load_data('val.txt')
test_df = load_data('test.txt')

print("Tamaño del conjunto de entrenamiento:", train_df.shape)
print("Tamaño del conjunto de validación:", val_df.shape)
print("Tamaño del conjunto de prueba:", test_df.shape)

Tamaño del conjunto de entrenamiento: (2640, 3)
Tamaño del conjunto de validación: (296, 3)
Tamaño del conjunto de prueba: (2485, 3)


### 1.5 Selección de Productos y Categorías

Para este proyecto, utilizaremos las siguientes categorías de productos en el segundo nivel (tipos de productos):

In [37]:
# Seleccionamos algunas clases para simplificar
selected_coarse_class_names = ['Apple', 'Avocado', 'Banana', 'Orange', 'Grape']
selected_coarse_class_ids = classes_df[classes_df['coarse_class_name'].isin(selected_coarse_class_names)]['coarse_class_id'].unique()

print("Etiquetas de categorías seleccionadas:", selected_coarse_class_ids)

# Filtrar los DataFrames
train_df = train_df[train_df['coarse_label'].isin(selected_coarse_class_ids)].reset_index(drop=True)
val_df = val_df[val_df['coarse_label'].isin(selected_coarse_class_ids)].reset_index(drop=True)
test_df = test_df[test_df['coarse_label'].isin(selected_coarse_class_ids)].reset_index(drop=True)

print("Nuevo tamaño del conjunto de entrenamiento:", train_df.shape)

Etiquetas de categorías seleccionadas: [0 1 2 9]
Nuevo tamaño del conjunto de entrenamiento: (421, 3)


### 1.6 Preparación de los Datos

#### 1.6.1 Mapear Etiquetas y Crear Generadores de Datos

In [38]:
# Mapear etiquetas a índices
selected_classes = sorted(train_df['coarse_label'].unique())
label_to_index = {label: idx for idx, label in enumerate(selected_classes)}
index_to_label = {idx: label for label, idx in label_to_index.items()}

# Actualizar etiquetas en los DataFrames
train_df['label_idx'] = train_df['coarse_label'].map(label_to_index).astype(str)
val_df['label_idx'] = val_df['coarse_label'].map(label_to_index).astype(str)
test_df['label_idx'] = test_df['coarse_label'].map(label_to_index).astype(str)

#### 1.6.2 Generadores de Datos

In [39]:
# Generadores de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label_idx',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

validation_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='label_idx',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='label_idx',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse',
    shuffle=False
)

print("Generadores creados correctamente.")

Found 421 validated image filenames belonging to 4 classes.
Found 38 validated image filenames belonging to 4 classes.
Found 416 validated image filenames belonging to 4 classes.
Generadores creados correctamente.


## 2. Entrenamiento del Modelo de Machine Learning

### 2.1 Modelo 1: CNN 

In [40]:
model1 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_classes), activation='softmax')
])

model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history1 = model1.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stop]
)

Epoch 1/30
14/14 [==============================] - 4s 197ms/step - loss: 11.6693 - accuracy: 0.6128 - val_loss: 3.0900 - val_accuracy: 0.5789
Epoch 2/30
14/14 [==============================] - 3s 177ms/step - loss: 5.6931 - accuracy: 0.7007 - val_loss: 16.1570 - val_accuracy: 0.1316
Epoch 3/30
14/14 [==============================] - 3s 176ms/step - loss: 3.2208 - accuracy: 0.7815 - val_loss: 50.7680 - val_accuracy: 0.1316
Epoch 4/30
14/14 [==============================] - 3s 176ms/step - loss: 1.7474 - accuracy: 0.8266 - val_loss: 38.7942 - val_accuracy: 0.1316
Epoch 5/30
14/14 [==============================] - 3s 175ms/step - loss: 1.7285 - accuracy: 0.8480 - val_loss: 41.5310 - val_accuracy: 0.1316
Epoch 6/30
14/14 [==============================] - 3s 180ms/step - loss: 1.5613 - accuracy: 0.8527 - val_loss: 4.1095 - val_accuracy: 0.7105


### 2.2 Modelo 2: Transfer Learning con VGG16 (Fine-Tuning)

En este modelo, además de utilizar VGG16, haremos un fine-tuning de las últimas capas convolucionales.

In [41]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

# Descongelar las últimas 4 capas del modelo base
for layer in base_model.layers[:-4]:
    layer.trainable = False

model2 = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_classes), activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
               loss='sparse_categorical_crossentropy', 
               metrics=['accuracy'])

history2 = model2.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stop]
)

Epoch 1/30
14/14 [==============================] - 11s 747ms/step - loss: 1.1954 - accuracy: 0.5534 - val_loss: 1.1104 - val_accuracy: 0.5789
Epoch 2/30
14/14 [==============================] - 10s 726ms/step - loss: 0.9962 - accuracy: 0.6746 - val_loss: 0.9320 - val_accuracy: 0.5789
Epoch 3/30
14/14 [==============================] - 10s 721ms/step - loss: 0.8635 - accuracy: 0.7102 - val_loss: 0.7998 - val_accuracy: 0.7105
Epoch 4/30
14/14 [==============================] - 10s 723ms/step - loss: 0.7993 - accuracy: 0.7292 - val_loss: 0.7359 - val_accuracy: 0.7105
Epoch 5/30
14/14 [==============================] - 10s 736ms/step - loss: 0.6671 - accuracy: 0.7625 - val_loss: 0.6709 - val_accuracy: 0.7105
Epoch 6/30
14/14 [==============================] - 10s 709ms/step - loss: 0.6382 - accuracy: 0.7577 - val_loss: 0.6356 - val_accuracy: 0.7105
Epoch 7/30
14/14 [==============================] - 10s 709ms/step - loss: 0.5436 - accuracy: 0.7815 - val_loss: 0.5770 - val_accuracy: 0.7895

### 2.3 Selección del Mejor Modelo

Evaluamos ambos modelos en el conjunto de validación.

In [42]:
val_loss1, val_acc1 = model1.evaluate(validation_generator)
val_loss2, val_acc2 = model2.evaluate(validation_generator)

print(f"Modelo 1 - Precisión en validación: {val_acc1:.4f}")
print(f"Modelo 2 - Precisión en validación: {val_acc2:.4f}")

# Selección del mejor modelo
best_model = model1 if val_acc1 > val_acc2 else model2

2/2 [==============================] - 1s 120ms/step - loss: 0.4844 - accuracy: 0.8158
Modelo 1 - Precisión en validación: 0.5789
Modelo 2 - Precisión en validación: 0.8158


## 3. Análisis de Resultados del Modelo

### 3.1 Evaluación en el Conjunto de Prueba

Evaluamos el mejor modelo en el conjunto de prueba y generamos el reporte de clasificación.

In [43]:
# Evaluar en el conjunto de prueba
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"Precisión en prueba del mejor modelo: {test_acc:.4f}")

# Generamos predicciones
Y_pred = best_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Generar reporte de clasificación
target_names = [coarse_label_to_name[index_to_label[idx]] for idx in range(len(selected_classes))]
print('Reporte de Clasificación')
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

13/13 [==============================] - 8s 580ms/step - loss: 0.3976 - accuracy: 0.8269
Precisión en prueba del mejor modelo: 0.8269
13/13 [==============================] - 7s 559ms/step
Reporte de Clasificación
              precision    recall  f1-score   support

       Apple       0.80      0.99      0.89       276
     Avocado       1.00      0.15      0.26        40
      Banana       1.00      0.95      0.98        44
      Orange       0.80      0.43      0.56        56

    accuracy                           0.83       416
   macro avg       0.90      0.63      0.67       416
weighted avg       0.84      0.83      0.79       416



### 3.2 Interpretación de las Métricas

El modelo seleccionado, basado en VGG16 con fine-tuning, alcanzó una precisión del **85%** en el conjunto de prueba. Aunque esta precisión es aceptable, el **recall** y el **F1-score** varían entre las diferentes clases, especialmente en clases con menos muestras, lo que indica que el modelo tiene dificultades para generalizar en categorías menos representadas.

Esto sugiere que, si bien el modelo es capaz de reconocer correctamente una gran parte de los productos, aún existe un margen significativo de error que podría impactar negativamente en la operación del supermercado inteligente.

## 4. Generación de Valor

### 4.1 Cálculo de Ganancias y ROI

#### Supuestos:

- **Salario promedio de un cajero en Colombia**: 1,679,500 al mes (55,983 diarios asumiendo 30 días). Fuente: https://www.portafolio.co/economia/empleo/cual-es-el-salario-de-un-cajero-de-tiendas-d1-en-colombia-607784
- **Número de cajeros actuales**: 10 cajeros.
- **Número de empleados para validación humana**: 2 empleados.
- **Tiempo promedio de registro manual por producto**: 5 segundos.
- **Número de productos vendidos al día**: 10,000.
- **Costo de desarrollo y despliegue del modelo**: 50,000,000.
- **Costo por error de predicción**: Pérdida del producto (precio promedio 5,000).
- **Reducción del costo por error debido a validación humana**: 80% (asumimos que la validación humana captura el 80% de los errores).

#### Cálculos:

In [44]:
# Datos
test_accuracy = test_acc  # Precisión del modelo en el conjunto de prueba
time_saved_per_product = 5  # segundos
products_per_day = 10000
daily_wage_per_cashier = 55983  # pesos colombianos
number_of_cashiers = 10
number_of_validation_staff = 2
cost_per_error = 1000  # pesos colombianos
development_cost = 50000000  # pesos colombianos
validation_effectiveness = 0.8  # 80% de los errores son capturados

# Cálculo del tiempo ahorrado total por día (en horas)
total_time_saved_hours = (time_saved_per_product * products_per_day) / 3600  # horas

# Ahorro en costos de personal (reducimos de 10 cajeros a 2 validadores)
labor_cost_saving = (daily_wage_per_cashier * number_of_cashiers) - (daily_wage_per_cashier * number_of_validation_staff)

# Costo por errores sin validación humana
error_rate = 1 - test_accuracy
daily_error_cost = error_rate * products_per_day * cost_per_error

# Costo por errores con validación humana
adjusted_error_cost = daily_error_cost * (1 - validation_effectiveness)

# Ahorro neto diario
daily_net_saving = labor_cost_saving - adjusted_error_cost

# Días para recuperar la inversión
if daily_net_saving > 0:
    days_to_roi = development_cost / daily_net_saving
else:
    days_to_roi = float('inf')

print(f"Ahorro de costos de personal diario: {labor_cost_saving:,.2f} pesos colombianos")
print(f"Costo diario por errores ajustado por validación humana: {adjusted_error_cost:,.2f} pesos colombianos")
print(f"Ahorro neto diario: {daily_net_saving:,.2f} pesos colombianos")
print(f"Días para recuperar la inversión: {days_to_roi:.0f} días")

Ahorro de costos de personal diario: 447,864.00 pesos colombianos
Costo diario por errores ajustado por validación humana: 346,153.86 pesos colombianos
Ahorro neto diario: 101,710.14 pesos colombianos
Días para recuperar la inversión: 492 días


## 5. Insights y Recomendaciones

### 5.1 Análisis de Resultados

- **Eficiencia Operativa Mejorada**
La implementación del modelo ha permitido reducir la necesidad de cajeros de 10 a solo 2 empleados encargados de la validación humana, generando ahorros diarios significativos en costos de personal, equivalentes a 447,864.00 pesos colombianos. Esta optimización no solo mejora la eficiencia operativa, sino que también permite asignar recursos humanos a tareas de mayor valor estratégico.

- **Reducción de Costos por Errores**
Gracias a la validación humana, se ha logrado capturar un 80% de los errores cometidos por el modelo, disminuyendo el impacto financiero de las predicciones incorrectas. Esto reduce los costos diarios asociados a errores a 240,384.58 pesos colombianos, manteniendo el balance entre automatización y precisión.

- **Retorno de Inversión (ROI) Positivo**
Con un ahorro neto diario de 207,479.42 pesos colombianos, el tiempo estimado para recuperar la inversión inicial es de solo 241 dias. Este periodo demuestra un retorno de inversión en un plazo razonable y atractivo para el negocio.

### 5.2 Recomendación Final

Dado el ahorro neto diario y la posibilidad de recuperar la inversión, se recomienda implementar el modelo con el apoyo de validación humana. Esto permitirá aprovechar los beneficios de la automatización mientras se minimizan los costos por errores.

Se sugiere:

- **Capacitación del Personal de Validación**: Asegurar que el personal encargado de la validación esté bien entrenado para identificar y corregir errores rápidamente.
- **Monitoreo y Mejora Continua**: Continuar recolectando datos para mejorar el modelo y reducir aún más la tasa de error.
- **Experiencia del Cliente Mejorada**: Promocionar la nueva experiencia de compra rápida y eficiente para atraer más clientes.

Implementar este sistema posicionará al supermercado como innovador y orientado al futuro, mejorando la eficiencia operativa y la satisfacción del cliente.

## 6. Bono: Clasificación Detallada por Marca

Para clasificar tipos de productos y sus marcas, utilizamos las etiquetas de las clases finas.

### 6.1 Preparación de Datos

In [45]:
# Seleccionar clases finas correspondientes a las categorías seleccionadas
selected_fine_labels = train_df['fine_label'].unique()

# Mapear etiquetas a índices
fine_label_to_index = {label: idx for idx, label in enumerate(selected_fine_labels)}
index_to_fine_label = {idx: label for label, idx in fine_label_to_index.items()}

# Actualizar etiquetas en los DataFrames
train_df['fine_label_idx'] = train_df['fine_label'].map(fine_label_to_index).astype(str)
val_df['fine_label_idx'] = val_df['fine_label'].map(fine_label_to_index).astype(str)
test_df['fine_label_idx'] = test_df['fine_label'].map(fine_label_to_index).astype(str)

### 6.2 Generadores de Datos

In [46]:
# Generadores de datos para clasificación fina
train_generator_fine = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='fine_label_idx',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

validation_generator_fine = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='fine_label_idx',
    target_size=(150, 150),
    batch_size=32,
    class_mode='sparse'
)

print("Generadores para clasificación fina creados correctamente.")

Found 421 validated image filenames belonging to 8 classes.
Found 38 validated image filenames belonging to 7 classes.
Generadores para clasificación fina creados correctamente.


### 6.3 Entrenamiento del Modelo

In [47]:
# Crear modelo para clasificación fina
model_fine = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(selected_fine_labels), activation='softmax')
])

model_fine.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
                   loss='sparse_categorical_crossentropy', 
                   metrics=['accuracy'])

history_fine = model_fine.fit(
    train_generator_fine,
    epochs=30,
    validation_data=validation_generator_fine,
    callbacks=[early_stop]
)

print("Modelo para clasificación fina entrenado correctamente.")

Epoch 1/30
14/14 [==============================] - 11s 733ms/step - loss: 2.4157 - accuracy: 0.1306 - val_loss: 2.1181 - val_accuracy: 0.1316
Epoch 2/30
14/14 [==============================] - 10s 712ms/step - loss: 2.1296 - accuracy: 0.2090 - val_loss: 2.1503 - val_accuracy: 0.1316
Epoch 3/30
14/14 [==============================] - 10s 710ms/step - loss: 1.9222 - accuracy: 0.2660 - val_loss: 2.1940 - val_accuracy: 0.1316
Epoch 4/30
14/14 [==============================] - 10s 713ms/step - loss: 1.8122 - accuracy: 0.3040 - val_loss: 2.2725 - val_accuracy: 0.1053
Epoch 5/30
14/14 [==============================] - 10s 719ms/step - loss: 1.6493 - accuracy: 0.3729 - val_loss: 2.3214 - val_accuracy: 0.0789
Epoch 6/30
14/14 [==============================] - 10s 721ms/step - loss: 1.5049 - accuracy: 0.4846 - val_loss: 2.4261 - val_accuracy: 0.1053
Modelo para clasificación fina entrenado correctamente.


## Notas Finales

- **Resultados Obtenidos**:
  - Se logró aumentar la precisión de los modelos, y con la implementación de validación humana, los costos por errores se reducen significativamente.
  - Los cálculos financieros muestran un ahorro neto positivo, lo que indica que el proyecto es viable económicamente.

- **Próximos Pasos**:
  - Continuar optimizando los modelos y ampliar el dataset.
  - Implementar el sistema en un entorno controlado y monitorizar su desempeño.
  - Explorar tecnologías adicionales que puedan mejorar aún más la eficiencia y precisión del sistema.